In [ ]:
!pip install requests beautifulsoup4 konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 19.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 488.6/488.6 kB 20.2 MB/s eta 0:00:00


In [ ]:
import requests
from bs4 import BeautifulSoup

url = 'https://www.kebhana.com/cont/customer/customer01/index.jsp'
headers = {'User-Agent': 'Mozilla/5.0'}
response = requests.get(url, headers=headers)
soup = BeautifulSoup(response.text, 'html.parser')

print(soup.prettify())

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html lang="ko" xml:lang="ko" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="No-cache" http-equiv="Cache-control"/>
  <meta content="No-cache" http-equiv="Pragma"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <title>
   자주하는 질문(FAQ) &lt; 고객센터  &lt; 하나은행
  </title>
  <!-- jquery -->
  <script src="https://image.kebhana.com/pbk/resource/js/lib/jquery.js" type="text/javascript">
  </script>
  <script type="text/javascript">
   var $j = jQuery.noConflict();
  </script>
  <script type="text/javascript">
   window._P1_CAUTH_LOG = 'Skit7fM/Kgrf+VbwKkaMMA==:655e04a1-c282-4224-8c8b-244c6dcd709e:1730972034393';
  </script>
  <script src="https://image.kebhana.com/pbk/resource/js/lib/wc_tagging_script.js" type="text/javascript">
  </script>
  <script src="https://image.kebhana

In [ ]:
import re
import requests
from bs4 import BeautifulSoup
import json

base_url = 'https://www.kebhana.com'
headers = {'User-Agent': 'Mozilla/5.0'}

faq_list = []

def get_faq_from_page(page_url, page_num):
    response = requests.get(base_url + page_url, headers=headers)
    soup = BeautifulSoup(response.text, 'html.parser')

    faq_container = soup.find('ul', class_='faqList')

    if not faq_container:
        print(f"[Page {page_num}] FAQ 데이터가 없습니다.")
        return

    for item in faq_container.find_all('li'):
        question_tag = item.find('a')
        answer_div = item.find('div', class_='answer')

        if not question_tag:
            continue

        raw_question = question_tag.get_text(separator=" ", strip=True)

        cleaned_question = re.sub(r'^(Q\.|질문|[\[\]\s]*)', '', raw_question).strip()

        if answer_div:
            answer_parts = set()

            for text_element in answer_div.find_all(['p', 'h5', 'div'], recursive=False):
                text = text_element.get_text(strip=True)
                if text:
                    answer_parts.add(text)

            for ul in answer_div.find_all('ul', recursive=False):
                for li in ul.find_all('li'):
                    text = li.get_text(strip=True)
                    if text:
                        answer_parts.add(text)

            table = answer_div.find('table')
            if table:
                for row in table.find_all('tr', recursive=False):
                    cols = [col.get_text(strip=True) for col in row.find_all(['td', 'th'], recursive=False)]
                    if cols:
                        answer_parts.add(" | ".join(cols))

            answer = "\n".join(sorted(answer_parts))
            faq_list.append({'question': cleaned_question, 'answer': answer})

def get_all_pages(total_pages):
    for page_num in range(1, total_pages + 1):
        if page_num == 1:
            page_url = '/cont/customer/customer01/index.jsp'
        else:
            page_url = f'/cont/customer/customer01/index,1,list,{page_num}.jsp'

        get_faq_from_page(page_url, page_num)

total_pages = 41
get_all_pages(total_pages)

output_file = '/content/faq_data_final.json'
with open(output_file, 'w', encoding='utf-8') as f:
    json.dump(faq_list, f, ensure_ascii=False, indent=4)

print(f"총 {len(faq_list)}개의 FAQ 데이터가 {output_file}에 저장되었습니다.")

총 405개의 FAQ 데이터가 /content/faq_data_final.json에 저장되었습니다.
